# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
!ls

CONTRIBUTING.md
LICENSE.md
README.md
env
homepage_actions.csv
index.ipynb
splitter.py


In [2]:
import pandas as pd
import numpy as np 
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('homepage_actions.csv')
df.head(15)

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [4]:
#Your code here
#Create a pivot table with user counts by group and action
pivot = df.pivot_table(index='group', columns='action', values='id', aggfunc='count', fill_value=0)

#Add a click-through rate column
pivot['click_rate'] = pivot['click'] / pivot['view']
pivot



action,click,view,click_rate
group,,,
control,932,3332,0.279712
experiment,928,2996,0.309746


In [5]:
#split the dataframe
control_df = df[df['group'] == 'control']
experiment_df = df[df['group'] == 'experiment']

#for control
n_ctrl = control_df['id'].nunique()  # Total unique users in control
clicks_ctrl = control_df[control_df['action'] == 'click']['id'].nunique()
ctr_ctrl = clicks_ctrl / n_ctrl

#for exeprinmental
n_exp = experiment_df['id'].nunique()
clicks_exp = experiment_df[experiment_df['action'] == 'click']['id'].nunique()


In [10]:
from statsmodels.stats.proportion import proportions_ztest

clicks = [clicks_exp, clicks_ctrl]
nobs = [n_exp, n_ctrl]

z_score, p_value = proportions_ztest(count=clicks, nobs=nobs, alternative='larger')


print(f"Z-score: {z_score:.4f}")
print(f"P-value: {p_value:.4f}")

Z-score: 2.6186
P-value: 0.0044


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [7]:
#Your code here
expected_clicks_exp = n_exp * ctr_ctrl

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [8]:
#Your code here
std_exp = np.sqrt(n_exp * ctr_ctrl * (1 - ctr_ctrl))


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [9]:
#Your code here

z_manual = (clicks_exp - expected_clicks_exp) / std_exp


p_manual = 1 - stats.norm.cdf(z_manual)
print(f"Manual Z-score: {z_manual:.4f}")
print(f"Manual P-value: {p_manual:.4f}")

Manual Z-score: 3.6625
Manual P-value: 0.0001


### Analysis:

Does this result roughly match that of the previous statistical test?

No: first instance
Z-score: 2.6186
P-value: 0.0044
: second instance
Manual Z-score: 3.6625
Manual P-value: 0.0001

new homepage design led to a statistically significant increase in clicks compared to the control.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.